<a href="https://colab.research.google.com/github/Thuan20146215/AI/blob/main/FinalPJ_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [15]:
df=pd.read_csv('train.csv')
df=df.dropna()###Drop Nan Values
X=df.drop('label',axis=1)
#X=df['title']
y=df['label']

In [12]:
y.head(10)

0     1
1     0
2     1
3     1
4     1
5     0
7     0
9     0
10    0
11    0
Name: label, dtype: int64

In [13]:
### Vocabulary size
voc_size=5000

In [16]:
messages=X.copy()
messages['title'][1]
messages.reset_index(inplace=True)

In [ ]:
import nltk
import re
from nltk.corpus import stopwords

In [17]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [18]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
13285
13286
13287
13288
13289
13290
13291
13292
13293
13294
13295
13296
13297
13298
13299
13300
13301
13302
13303
13304
13305
13306
13307
13308
13309
13310
13311
13312
13313
13314
13315
13316
13317
13318
13319
13320
13321
13322
13323
13324
13325
13326
13327
13328
13329
13330
13331
13332
13333
13334
13335
13336
13337
13338
13339
13340
13341
13342
13343
13344
13345
13346
13347
13348
13349
13350
13351
13352
13353
13354
13355
13356
13357
13358
13359
13360
13361
13362
13363
13364
13365
13366
13367
13368
13369
13370
13371
13372
13373
13374
13375
13376
13377
13378
13379
13380
13381
13382
13383
13384
13385
13386
13387
13388
13389
13390
13391
13392
13393
13394
13395
13396
13397
13398
13399
13400
13401
13402
13403
13404
13405
13406
13407
13408
13409
13410
13411
13412
13413
13414
13415
13416
13417
13418
13419
13420
13421
13422
13423
13424
13425
13426
13427
13428
13429
13430
13431
13432
13433
13434
13435
13436
13437
13438
13439
13440
13441
1

In [19]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[2981, 3576, 2191, 1893, 276, 3071, 514, 3809, 2862, 3129],
 [1933, 2758, 4797, 1743, 4329, 3599, 2162],
 [2737, 3814, 3107, 2321],
 [4398, 4345, 619, 4888, 2198, 2246],
 [1975, 4329, 4986, 4571, 2146, 2908, 4329, 1435, 3858, 4135],
 [180,
  921,
  4947,
  2939,
  1293,
  2396,
  359,
  4233,
  4889,
  2593,
  1773,
  4916,
  2488,
  4076,
  2162],
 [2430, 3675, 3976, 4424, 4480, 1202, 1657, 2782, 3174, 4431, 1706],
 [1454, 850, 3845, 2031, 765, 4772, 2396, 4230, 3174, 4431, 1706],
 [2302, 3898, 1105, 1233, 4436, 3659, 261, 4397, 2396, 3315],
 [4061, 148, 2689, 3874, 1441, 4826, 852, 278],
 [3531, 1310, 1334, 3804, 3432, 1694, 614, 516, 3874, 3454, 380],
 [4888, 1560, 276, 3659, 2396, 765],
 [2535, 552, 697, 2984, 1379, 3973, 4515, 3784, 4869],
 [891, 4014, 4995, 4395, 3123, 3745, 2808, 3174, 4431, 1706],
 [1095, 3212, 1214, 3877, 4350, 3174, 4431, 1706],
 [486, 3754, 1834, 797, 2748, 3929, 3683, 4279, 1962, 1632],
 [622, 1790, 2758],
 [3196, 3096, 4187, 3125, 2396, 808, 4034, 2162],


In [20]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 3809 2862 3129]
 [   0    0    0 ... 4329 3599 2162]
 [   0    0    0 ... 3814 3107 2321]
 ...
 [   0    0    0 ... 3174 4431 1706]
 [   0    0    0 ... 3219 2642 4206]
 [   0    0    0 ... 4634 2766  401]]


In [ ]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,  463,
       3694,  442, 1422, 4560,  794, 2228, 4644, 1886, 3993], dtype=int32)

In [21]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dense(100,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 dropout (Dropout)           (None, 20, 40)            0         
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 100)               10100     
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 266,601
Trainable params: 266,601
Non-trai

In [ ]:
len(embedded_docs),y.shape

(18285, (18285,))

In [22]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [23]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [24]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.3, random_state=42)

In [25]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
200/200 [==============================] - 13s 45ms/step - loss: 0.3015 - accuracy: 0.8619 - val_loss: 0.1971 - val_accuracy: 0.9151
Epoch 2/10
200/200 [==============================] - 10s 51ms/step - loss: 0.1479 - accuracy: 0.9417 - val_loss: 0.2114 - val_accuracy: 0.9196
Epoch 3/10
200/200 [==============================] - 7s 38ms/step - loss: 0.1065 - accuracy: 0.9625 - val_loss: 0.2121 - val_accuracy: 0.9174
Epoch 4/10
200/200 [==============================] - 16s 78ms/step - loss: 0.0806 - accuracy: 0.9722 - val_loss: 0.2373 - val_accuracy: 0.9174
Epoch 5/10
200/200 [==============================] - 11s 56ms/step - loss: 0.0581 - accuracy: 0.9810 - val_loss: 0.2774 - val_accuracy: 0.9125
Epoch 6/10
200/200 [==============================] - 8s 39ms/step - loss: 0.0431 - accuracy: 0.9852 - val_loss: 0.3244 - val_accuracy: 0.9171
Epoch 7/10
200/200 [==============================] - 15s 73ms/step - loss: 0.0307 - accuracy: 0.9899 - val_loss: 0.4152 - val_accuracy: 0

In [26]:
y_pred=(model.predict(X_test) > 0.5).astype("int32")

from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

172/172 [==============================] - 2s 6ms/step


0.907765220561429

In [ ]:
print(y_test[:10])
print(y_pred[:10])

[1 0 0 0 1 1 0 1 1 1]
[[1]
 [0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]]


In [ ]:
##chay thu
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus1 = []
text = input('Enter Title: ')
for i in range(0, len(text)):
    
    review = re.sub('[^a-zA-Z]', ' ', text)
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus1.append(review)

onehot_repr1=[one_hot(words,voc_size)for words in corpus1] 
embedded_docs1=pad_sequences(onehot_repr1,padding='pre',maxlen=20)
test = np.array(embedded_docs1)
y_pred=(model.predict(test) > 0.5).astype("int32")
if (y_pred[0]==1):
  print('Fake New !!!')
else:
  print('Real New !!!')

Enter Title: Texas Oil Fields Rebound From Price Lull, but Jobs Are Left Behind - The New York Times
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
3/3 [==============================] - 0s 9ms/step
Real New !!!


In [27]:
from nltk.stem.porter import PorterStemmer
def runtest(text):
  ps = PorterStemmer()
  corpus1 = []
  for i in range(0, len(text)):
     
      review = re.sub('[^a-zA-Z]', ' ', text)
      review = review.lower()
      review = review.split()
      
      review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
      review = ' '.join(review)
      corpus1.append(review)

  onehot_repr1=[one_hot(words,voc_size)for words in corpus1] 
  embedded_docs1=pad_sequences(onehot_repr1,padding='pre',maxlen=20)
  test = np.array(embedded_docs1)
  y_pred=(model.predict(test) > 0.5).astype("int32")
  return y_pred[0]


In [28]:
text = input('Enter Title: ')
x = runtest(text)

if (x==1):
  print(text + ' is a Fake New !!!')
else:
  print(text + ' is a Real New !!!')

Enter Title: Trump Campaign Says Hillary Supporter Tried Assassinating Trump â€“ It Was A Republican With A Poster
4/4 [==============================] - 0s 6ms/step
Trump Campaign Says Hillary Supporter Tried Assassinating Trump â€“ It Was A Republican With A Poster is a Fake New !!!


In [ ]:
model.save("/content/drive/MyDrive/ai/FnPJ2.h5")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')